In [29]:
import os
from tqdm.notebook import tqdm
import polars as pl
import plotly.graph_objects as go
import json
from datetime import datetime



In [30]:
from plot_utils_2 import get_plotly_layout
from plot_utils_2 import colors
width, height = 1000, 450

In [31]:
# Existing dataset dir
data_dir = './data/'

# Existing plots dir
plots_dir = data_dir+'/plots/'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)


In [32]:
plot_settings = {
    'arbitrum': {'color': colors['blue'],   'label': 'Arbitrum', 'style': 'solid', 'width': 4, 'marker_symbol': 'circle'},
    'arbitrum2': {'color': colors['blue'],   'label': 'Arbitrum', 'style': 'solid', 'width': 4, 'marker_symbol': 'circle'},
    'base': {'color': colors['red'],        'label': 'Base', 'style': 'dash', 'width': 3.5, 'marker_symbol': 'diamond'},
    'ethereum': {'color': colors['green'],  'label': 'Ethereum', 'style': 'dot', 'width': 3, 'marker_symbol': 'square'},
    'optimism': {'color': colors['grey'],   'label': 'Optimism', 'style': 'dashdot', 'width': 2.5, 'marker_symbol': 'triangle-up'},
    'zksync': {'color': colors['pink'],     'label': 'zkSync', 'style': 'solid', 'width': 2, 'marker_symbol': 'cross'},
    'arbitrum2': {'color': colors['brown'],    'label': 'Fantom', 'style': 'dash', 'width': 1.5, 'marker_symbol': 'star'},
}

file_settings = {
    'ethereum': {  'nr': '1', 'file_name': './data/mav-ethereum-Uniswap-v3-WETH-USDC.paraquet' , 'output_file': './data/mav2-ethereum-Uniswap-v3-WETH-USDC.paraquet' },
    'arbitrum': {  'nr': '2', 'file_name': './data/mav-arbitrum-Uniswap-v3-WETH-USDC.paraquet' , 'output_file': './data/mav2-arbitrum-Uniswap-v3-WETH-USDC.paraquet'},
    'arbitrum2': { 'nr': '3', 'file_name': './data/mav-arbitrum-Uniswap-v3-WETH-USDCe.paraquet', 'output_file': './data/mav2-arbitrum-Uniswap-v3-WETH-USDCe.paraquet'},
    'base': {      'nr': '4', 'file_name': './data/mav-base-Uniswap-v3-WETH-USDC.paraquet'     , 'output_file': './data/mav2-base-Uniswap-v3-WETH-USDC.paraquet'},
    'optimism': {  'nr': '5', 'file_name': './data/mav-optimism-Uniswap-v3-WETH-USDC.paraquet' , 'output_file': './data/mav2-optimism-Uniswap-v3-WETH-USDC.paraquet'},
    'zksync': {    'nr': '6', 'file_name': './data/mav-zksync-Uniswap-v3-WETH-USDC.paraquet'   , 'output_file': './data/mav2-zksync-Uniswap-v3-WETH-USDC.paraquet' } ,
}



In [33]:
chains = [
    #'ethereum',
    #'arbitrum',
    #'arbitrum2',
    #'base',
    'optimism',
    #'zksync'
]


In [34]:
swap_amounts = {}  # Dictionary to store tx count data for each chain

for chain in chains:

    file_name = file_settings[chain]['file_name']
    swaps_df = pl.scan_parquet(file_name).collect(streaming=True)

    datetime_threshold = datetime.strptime('2024-03-13', '%Y-%m-%d')

    # Create the 'blobs' column based on the condition
    swaps_df = swaps_df.with_columns(
        (pl.col('block_timestamp') > datetime_threshold).alias('blobs')
    )

    # Select 'amount_USD' and 'blobs' columns
    swaps_df = swaps_df.select(['amount_USD', 'blobs'])

    # Calculate Q1, Q3, and IQR
    Q1 = swaps_df['amount_USD'].quantile(0.25)
    Q3 = swaps_df['amount_USD'].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for outlier removal
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    swaps_df = swaps_df.filter(
        (pl.col('amount_USD') >= lower_bound) & (pl.col('amount_USD') <= upper_bound)
    )

    swap_amounts[chain] = swaps_df#swaps_df['amount_USD']

#print(swap_amounts['zksync'])


In [35]:


# Plot the violin plot
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

# Add traces for both DataFrames
for chain in chains:
    fig.add_trace(go.Violin(y=swap_amounts[chain], box_visible=True, line_color=plot_settings[chain]['color'], name=plot_settings[chain]['label']))

fig.update_layout(
    title="Distribution of Swap Volume (USD)",
    showlegend=False,
    yaxis_title="Volume (USD)",
    yaxis_tickformat='.23'#'.2f'
)

# Save the plot as a PDF file
fig.write_image(plots_dir + "amount_usd_distribution_violin_"+chain+".pdf")

# Show the plot
fig.show()


ValueError: Transform failed with error code 525: Cannot read property '0' of undefined

In [ ]:
# Plot the violin plot
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

# Add traces for each chain and 'blobs' value
for chain in chains:
    # Extract 'amount_USD' values where 'blobs' is True or False
    amounts_yes = swap_amounts[chain].filter(pl.col('blobs')==True)['amount_USD']
    amounts_no = swap_amounts[chain].filter(pl.col('blobs')==False)['amount_USD']


    # Add traces for 'blobs=yes' and 'blobs=no'
    fig.add_trace(go.Violin(y=amounts_yes, box_visible=True, line_color=plot_settings[chain]['color'],
                            name=f"{plot_settings[chain]['label']} (blobs=yes)"))
    fig.add_trace(go.Violin(y=amounts_no, box_visible=True, line_color=plot_settings[chain]['color'],
                            name=f"{plot_settings[chain]['label']} (blobs=no)"))

# Update layout
fig.update_layout(
    title="Distribution of Swap Volume (USD) by Blobs",
    yaxis_title="Volume (USD)",
    showlegend=False,  # Show legend with labels
    yaxis_tickformat='.2f'  # Adjust tick format as needed
)

# Save the plot as a PDF file
fig.write_image(f"{plots_dir}amount_usd_distribution_violin"+chain+".pdf")

# Show the plot
fig.show()

In [ ]:
swap_df = swap_amounts[chain]
swap_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the violin plot
plt.figure(figsize=(width, height))
#sns.violinplot(x='Chain', y='Amount USD', data=swap_amounts[chain], palette={plot_settings[chain]['label']: plot_settings[chain]['color'] for chain in chains})

sns.violinplot(data=swap_df, y="amount_USD")


# Update plot title and axis labels
plt.title("Distribution of Swap Volume (USD)")
plt.yscale('log')  # Optional: Change the y-axis to log scale for better visualization
plt.ylabel("Volume (USD)")

# Save the plot as a PDF file
plt.savefig(f"{plots_dir}amount_usd_distribution_violin.pdf", format="pdf")

# Show the plot
plt.show()




#for chain in chains:
#    fig.add_trace(go.Violin(y=swap_amounts[chain], box_visible=True, line_color=plot_settings[chain]['color'], name=plot_settings[chain]['label']))


In [ ]:
ewre rwe
del swap_amounts

In [ ]:
import polars as pl
import plotly.graph_objects as go

# Assuming chains, file_settings, and plot_settings are already defined

# Dictionary to store pre-processed and aggregated amount_usd data
processed_amounts = {}

for chain in chains:
    file_name = file_settings[chain]['file_name']
    
    # Read and collect data using Polars
    swaps_df = pl.scan_parquet(file_name).collect()
    
    # Calculate Q1, Q3, and IQR
    Q1 = swaps_df['amount_USD'].quantile(0.25)
    Q3 = swaps_df['amount_USD'].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for outlier removal
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    filtered_swaps_df = swaps_df.filter(
        (pl.col('amount_USD') >= lower_bound) & (pl.col('amount_USD') <= upper_bound)
    )
    
    # Bin the amount_USD values by 1.0
    bins = pl.cut(filtered_swaps_df['amount_USD'], bins=range(int(filtered_swaps_df['amount_USD'].min()), int(filtered_swaps_df['amount_USD'].max()) + 2), labels=False)
    
    # Aggregate the data by bins
    binned_data = filtered_swaps_df.with_column(bins.alias('bins')).groupby('bins').agg([
        pl.col('amount_USD').count().alias('count')
    ])
    
    # Convert to list and store in dictionary
    processed_amounts[chain] = binned_data.to_dict(as_series=False)

# Define the layout dimensions
width = 800
height = 600
plots_dir = './'  # Specify your plots directory

# Create the figure with optimized layout
fig = go.Figure(layout=go.Layout(width=width, height=height))

# Add traces for each chain
for chain in chains:
    binned_data = processed_amounts[chain]  # Access pre-processed and aggregated data
    settings = plot_settings[chain]        # Access plot settings
    fig.add_trace(go.Bar(
        x=binned_data['bins'],
        y=binned_data['count'],
        marker_color=settings['color'],
        name=settings['label']
    ))

# Update layout to hide legend and format y-axis to scientific notation
fig.update_layout(
    title="Distribution of Swap Volume (USD)",
    showlegend=False,
    xaxis_title="Volume (USD) Bins",
    yaxis_title="Count",
    yaxis_tickformat='.2e'  # Scientific notation
)

# Save the plot as a PDF file
fig.write_image(f"{plots_dir}amount_usd_distribution_bar.pdf")

# Show the plot
fig.show()


In [ ]:
# Assuming swaps_df1 and swaps_df2 are already defined and are Polars DataFrames

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df.filter((df[column] >= lower_bound) & (df[column] <= upper_bound))

# Remove outliers from both DataFrames
swaps_df1 = remove_outliers(swaps_df1, 'amount_USD')
swaps_df2 = remove_outliers(swaps_df2, 'amount_USD')

In [ ]:
sfdd fwedfa

In [ ]:
file_name = "./data/swaps-base-Uniswap-v3-WETH-USDC.paraquet"
swaps_df = pl.scan_parquet(file_name).collect(streaming=True)
swaps_df1 = swaps_df

In [ ]:
swaps_df['amount_USD']

In [ ]:
# Calculate the first and third quartiles (Q1 and Q3) for 'amount_USD'
Q1 = swaps_df['amount_USD'].quantile(0.25)
Q3 = swaps_df['amount_USD'].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
cleaned_swaps_df = swaps_df.filter((swaps_df['amount_USD'] >= lower_bound) & (swaps_df['amount_USD'] <= upper_bound))

# Convert the cleaned 'amount_USD' column to a list
amount_usd_list_cleaned = cleaned_swaps_df['amount_USD'].to_list()

# Define the layout dimensions
width = 800
height = 600
plots_dir = './'  # Specify your plots directory
chain = 'chain_name'  # Specify your chain name

# Plot the histogram
fig = go.Figure(layout=go.Layout(width=width, height=height))

fig.add_trace(go.Histogram(x=amount_usd_list_cleaned, marker_color='blue', name='Amount USD'))

fig.update_layout(
    title="Distribution of Amount USD (Without Outliers)",
    xaxis_title="Amount USD",
    yaxis_title="Count",
    bargap=0.2,
)

# Save the plot as a PDF file
fig.write_image(plots_dir + "amount_usd_distribution_" + chain + ".pdf")

# Show the plot
fig.show()


In [ ]:
file_name = "./data/swaps-ethereum-Uniswap-v3-WETH-USDC.paraquet"
swaps_df = pl.scan_parquet(file_name).collect(streaming=True)
swaps_df2 = swaps_df

In [ ]:

# Calculate the first and third quartiles (Q1 and Q3) for 'amount_USD'
Q1 = swaps_df['amount_USD'].quantile(0.25)
Q3 = swaps_df['amount_USD'].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
swaps_df = swaps_df.filter((swaps_df['amount_USD'] >= lower_bound) & (swaps_df['amount_USD'] <= upper_bound))

# Convert the cleaned 'amount_USD' column to a list
#amount_usd_list_cleaned = cleaned_swaps_df['amount_USD'].to_list()

# Define the layout dimensions
width = 800
height = 600
plots_dir = './'  # Specify your plots directory
chain = 'chain_name'  # Specify your chain name

# Plot the histogram
fig = go.Figure(layout=go.Layout(width=width, height=height))

fig.add_trace(go.Histogram(x=swaps_df['amount_USD'], marker_color='blue', name='Amount USD'))

fig.update_layout(
    title="Distribution of Amount USD (Without Outliers)",
    xaxis_title="Amount USD",
    yaxis_title="Count",
    bargap=0.2,
)

# Save the plot as a PDF file
fig.write_image(plots_dir + "amount_usd_distribution_" + chain + ".pdf")

# Show the plot
fig.show()


In [ ]:
# Define the layout dimensions
width = 800
height = 600
plots_dir = './'  # Specify your plots directory
chain = 'chain_name'  # Specify your chain name

# Plot the violin plot
fig = go.Figure(layout=go.Layout(width=width, height=height))

fig.add_trace(go.Violin(y=swaps_df['amount_USD'], box_visible=True, line_color='blue', name='Amount USD'))

fig.update_layout(
    title="Distribution of Amount USD (Without Outliers)",
    xaxis_title="Amount USD",
    yaxis_title="Density",
    yaxis_tickformat='.2f'
)

# Save the plot as a PDF file
fig.write_image(plots_dir + "amount_usd_distribution_violin_" + chain + ".pdf")

# Show the plot
fig.show()


In [ ]:


# Assuming swaps_df1 and swaps_df2 are already defined and are Polars DataFrames

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df.filter((df[column] >= lower_bound) & (df[column] <= upper_bound))

# Remove outliers from both DataFrames
swaps_df1 = remove_outliers(swaps_df1, 'amount_USD')
swaps_df2 = remove_outliers(swaps_df2, 'amount_USD')

# Define the layout dimensions
width = 800
height = 600
plots_dir = './'  # Specify your plots directory
chain = 'chain_name'  # Specify your chain name

# Plot the violin plot
fig = go.Figure(layout=go.Layout(width=width, height=height))

# Add traces for both DataFrames
fig.add_trace(go.Violin(y=swaps_df1['amount_USD'], box_visible=True, line_color='blue', name='Amount USD - DataFrame 1'))
fig.add_trace(go.Violin(y=swaps_df2['amount_USD'], box_visible=True, line_color='green', name='Amount USD - DataFrame 2'))

fig.update_layout(
    title="Distribution of Amount USD (Without Outliers)",
    xaxis_title="DataFrames",
    yaxis_title="Amount USD",
    yaxis_tickformat='.2f'
)

# Save the plot as a PDF file
fig.write_image(plots_dir + "amount_usd_distribution_violin_" + chain + ".pdf")

# Show the plot
fig.show()


In [ ]:
del swaps_df1
del swaps_df2

In [ ]:
#Top Operations
chain = chains[0]
data = inscriptions_groups_dict[chain]
value_counts = get_top_operations(data)

# Plot the bar chart
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

fig.add_trace(go.Bar(x=value_counts['op'], y=value_counts['count'],
              marker_color=colors['blue'], textposition='auto', text=value_counts['count'].round(2), name='Operation'))
fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Operation", yaxis_ticksuffix="%")

fig.update_traces(
    texttemplate='<b>%{text:,.4}</b>', textfont_size=18)
fig.update_yaxes(range=[0, 100])

fig.write_image(plots_dir+"top-15-operation-"+chain+".pdf")

fig.show()
